In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.utils import to_categorical
from tensorflow.examples.tutorials.mnist import input_data
import time

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
data = pd.read_csv('/content/gdrive/My Drive/DeepLearning/train_july.csv')

In [0]:
train_y = data['label']
train_X = data.drop(['label'],axis=1)

In [0]:
train_X = np.array(train_X)
train_y = np.array(train_y).reshape(60000,1)
train_y = to_categorical(train_y)

In [0]:
x = tf.placeholder('float',[None,784])
y = tf.placeholder('float',[None,10])
x_image = tf.reshape(x,[-1,28,28,1])

In [0]:
#第一层
filter1 = tf.Variable(tf.truncated_normal([5,5,1,6]))
bias1 = tf.Variable(tf.truncated_normal([6]))
conv1 = tf.nn.conv2d(x_image,filter1,strides=[1,1,1,1],padding='SAME')
h_conv1 = tf.nn.sigmoid(conv1+bias1)
#第二层
maxPool2 = tf.nn.max_pool(h_conv1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
#第三层
filter2 = tf.Variable(tf.truncated_normal([5,5,6,16]))
bias2 = tf.Variable(tf.truncated_normal([16]))
conv2 = tf.nn.conv2d(maxPool2,filter2,strides=[1,1,1,1],padding='SAME')
h_conv2 = tf.nn.sigmoid(conv2+bias2)
#第四层
maxPool3 = tf.nn.max_pool(h_conv2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
#第五层
filter3 = tf.Variable(tf.truncated_normal([5,5,16,120]))
bias3 = tf.Variable(tf.truncated_normal([120]))
conv3 = tf.nn.conv2d(maxPool3,filter3,strides=[1,1,1,1],padding='SAME')
h_conv3 = tf.nn.sigmoid(conv3)
#第六层
W_fc1 = tf.Variable(tf.truncated_normal([7*7*120,80]))
bias_fc1 = tf.Variable(tf.truncated_normal([80]))
h_pool2_flat = tf.reshape(h_conv3,[-1,7*7*120])
h_fc1 = tf.nn.sigmoid(tf.matmul(h_pool2_flat,W_fc1)+bias_fc1)
#第七层
W_fc2 = tf.Variable(tf.truncated_normal([80,10]))
bias_fc2 = tf.Variable(tf.truncated_normal([10]))
y_conv = tf.nn.softmax(tf.matmul(h_fc1,W_fc2)+bias_fc2)

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
cross_entropy = -tf.reduce_sum(y*tf.log(y_conv))
train_step = tf.train.GradientDescentOptimizer(0.001).minimize(cross_entropy)

In [0]:
sess = tf.InteractiveSession()
correct_prediction = tf.equal(tf.argmax(y_conv,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))
sess.run(tf.global_variables_initializer())

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [0]:
c = []
for j in range(150):
  for i in range(300):
      batch_xs = train_X[i*200:(i+1)*200,:]
      batch_xs = batch_xs/255
      batch_ys = train_y[i*200:(i+1)*200,:]
      train_step.run(feed_dict={x:batch_xs,y:batch_ys})
      if i %10 ==0:
          train_accuracy = accuracy.eval(feed_dict={x: batch_xs, y: batch_ys})
          c.append(train_accuracy)
          print("step %d, training accuracy %g"%(i,train_accuracy))
plt.show(c)

step 0, training accuracy 0.115
step 10, training accuracy 0.155
step 20, training accuracy 0.225
step 30, training accuracy 0.36
step 40, training accuracy 0.23
step 50, training accuracy 0.33
step 60, training accuracy 0.265
step 70, training accuracy 0.22
step 80, training accuracy 0.33
step 90, training accuracy 0.5
step 100, training accuracy 0.465
step 110, training accuracy 0.44
step 120, training accuracy 0.52
step 130, training accuracy 0.5
step 140, training accuracy 0.51
step 150, training accuracy 0.49
step 160, training accuracy 0.54
step 170, training accuracy 0.56
step 180, training accuracy 0.475
step 190, training accuracy 0.535
step 200, training accuracy 0.65
step 210, training accuracy 0.575
step 220, training accuracy 0.585
step 230, training accuracy 0.645
step 240, training accuracy 0.735
step 250, training accuracy 0.65
step 260, training accuracy 0.57
step 270, training accuracy 0.665
step 280, training accuracy 0.585
step 290, training accuracy 0.79
step 0, tr

In [0]:
test = pd.read_csv('/content/gdrive/My Drive/DeepLearning/test_july.csv')

In [0]:
test_X = test.drop(['index'],axis=1)
test_X = test_X/255

In [0]:
test_Y = tf.argmax(y_conv,1)

In [0]:
output = test_Y.eval(session=sess,feed_dict={x:test_X})

In [0]:
submit = pd.read_csv('/content/gdrive/My Drive/DeepLearning/sample_submission.csv')

In [0]:
submit['label']=output

In [0]:
submit.to_csv('sample_submission.csv',index=None)